In [1]:
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing.popen_spawn_win32

In [2]:
#start a local dask cluster
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:10360 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 34.27 GB


In [35]:
#read data
sys_info = dd.read_csv('D:/180B/data/system_sysinfo_unique_normalized.csv000.gz',
                       delimiter ="\1",
                       assume_missing=True)
sys_info.head()

c:\users\yxx726\appdata\local\programs\python\python36\lib\site-packages\dask\dataframe\io\csv.py:481: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  "Setting ``blocksize=None``" % compression


,load_ts,guid,chassistype,chassistype_2in1_category,countryname,countryname_normalized,modelvendor,modelvendor_normalized,model,model_normalized,...,cpu_suffix,screensize_category,persona,processor_line,vpro_enabled,firstreportdate,lastreportdate,discretegraphics,cpu_stepping,engagement_id
0,2020-10-01 00:48:15,10351643a4ae4b2d9cbc8a90db184050,Notebook,Unknown,Brazil,Brazil,Lenovo,Lenovo,Lenovo G480,Lenovo G480,...,Core-M,14x,Web User,M-Processor,N,2020-01-12 20:31:03,2020-09-29 18:30:56,N,Intel64 Family 6 Model 42 Stepping 7,iduu
1,2020-10-01 00:48:15,4f40b2f38cd74f6ea1559cebbc924619,Notebook,Unknown,Sweden,Sweden,Asus,Asus,T300CHI,T300CHI,...,Core-Y,12x,Web User,Unknown,N,2018-11-15 17:24:52,2020-09-28 12:49:29,N,Intel64 Family 6 Model 61 Stepping 4,iduu
2,2020-10-01 00:48:15,04b81eb4d661410ebff356f27fc544ed,Desktop,Unknown,Viet Nam,Viet Nam,System manufacturer,System manufacturer,System Product Name,System Product Name,...,Other,23x,Office/Productivity,Unknown,Y,2020-05-09 12:24:37,2020-09-29 08:54:35,Y,Intel64 Family 6 Model 158 Stepping 12,intel.com
3,2020-10-01 00:48:15,afd55a587fe14fc1904d572b3230dd54,Notebook,Unknown,Mexico,Mexico,HP,HP,HP ProBook 640 G2,HP ProBook 640 G2,...,Core-U,14x,Office/Productivity,U-Processor,N,2019-10-22 13:56:27,2020-09-28 08:41:47,N,Intel64 Family 6 Model 78 Stepping 3,intel.com
4,2020-10-01 00:48:15,4b3d6747c5f541949f4c1bd127b00bd9,Notebook,Unknown,Romania,Romania,Acer,Acer,Nitro AN515-52,Nitro AN515-52,...,Core-H,15x,Web User,H-Processor,N,2020-04-01 17:37:46,2020-04-23 19:34:16,Y,Intel64 Family 6 Model 158 Stepping 10,intel.com


In [12]:
sys_info.columns

Index(['load_ts', 'guid', 'chassistype', 'chassistype_2in1_category',
       'countryname', 'countryname_normalized', 'modelvendor',
       'modelvendor_normalized', 'model', 'model_normalized', 'ram', 'os',
       '#ofcores', 'age_category', 'graphicsmanuf', 'gfxcard',
       'graphicscardclass', 'processornumber', 'cpuvendor', 'cpuname',
       'cpucode', 'cpu_family', 'cpu_suffix', 'screensize_category', 'persona',
       'processor_line', 'vpro_enabled', 'firstreportdate', 'lastreportdate',
       'discretegraphics', 'cpu_stepping', 'engagement_id'],
      dtype='object')

# Paired T-tests

In [4]:
sys_info.persona.value_counts().compute()

Web User                 35153
Casual User              12810
Gamer                    10611
Casual Gamer             10498
Office/Productivity       7519
Content Creator/IT        5883
Communication             5836
Win Store App User        4968
Entertainment             3768
File & Network Sharer     2231
Unknown                    723
Name: persona, dtype: int64

In [58]:
import pandas as pd
import dask
import dask.dataframe as dd
import scipy.stats as stats

def hypo_test_on_numerical(datapath, feature):
    # This function finds all pvalues between each persona with the given feature
    
    data = dd.read_csv(datapath,
                       delimiter ="\1",
                       assume_missing=True)
    results = {}
    
    #list for all types of users
    all_persona = list(data.persona.unique().compute())
    all_persona.remove('Unknown')
    print(all_persona)
    # test between all personas
    for i in all_persona:
        # data of ram of persona i
        d1 = data.loc[data['persona'] == i][feature]
        for j in all_persona:
            if (i,j) not in results.keys() and (i !=j):
                # data of persona i
                d2 = data.loc[data['persona'] == j][feature]
                #testing
                output = stats.ttest_ind(a = d1, b = d2)
                #put into results dict
                results[(i,j)] = output.pvalue
            else:
                pass
                
    return results

def unsignificants(results_dict, significant_level):
    # This function finds all unsignificant values in the results list
    unsig = {}
    keys = results_dict.keys()
    for i in keys:
        if results_dict[i] > significant_level:
            unsig[i] = results_dict[i]
    return unsig

In [55]:
results = hypo_test_on_numerical('D:/180B/data/system_sysinfo_unique_normalized.csv000.gz', 'ram')

c:\users\yxx726\appdata\local\programs\python\python36\lib\site-packages\dask\dataframe\io\csv.py:481: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  "Setting ``blocksize=None``" % compression


['Web User', 'Office/Productivity', 'Casual Gamer', 'Entertainment', 'Communication', 'Casual User', 'Gamer', 'Content Creator/IT', 'Win Store App User', 'File & Network Sharer']


In [59]:
unsignificants(results,0.05)

{('Office/Productivity', 'Win Store App User'): 0.36636187121707253,
 ('Casual Gamer', 'Gamer'): 0.06209231165493901,
 ('Entertainment', 'Casual User'): 0.051284385452370934,
 ('Entertainment', 'File & Network Sharer'): 0.23231563197860566,
 ('Communication', 'Casual User'): 0.36435021350958885,
 ('Communication', 'File & Network Sharer'): 0.42337335566425727,
 ('Casual User', 'Entertainment'): 0.051284385452370934,
 ('Casual User', 'Communication'): 0.36435021350958885,
 ('Casual User', 'File & Network Sharer'): 0.8399339169962561,
 ('Gamer', 'Casual Gamer'): 0.06209231165493901,
 ('Win Store App User', 'Office/Productivity'): 0.36636187121707253,
 ('File & Network Sharer', 'Entertainment'): 0.23231563197860566,
 ('File & Network Sharer', 'Communication'): 0.42337335566425727,
 ('File & Network Sharer', 'Casual User'): 0.8399339169962561}

In [60]:
results

{('Web User', 'Office/Productivity'): 3.766131469002691e-06,
 ('Web User', 'Casual Gamer'): 5.583512828016116e-63,
 ('Web User', 'Entertainment'): 0.042265218754574924,
 ('Web User', 'Communication'): 2.2534434999585363e-11,
 ('Web User', 'Casual User'): 2.661729093536839e-13,
 ('Web User', 'Gamer'): 2.102470376967744e-48,
 ('Web User', 'Content Creator/IT'): 1.1590440486812365e-113,
 ('Web User', 'Win Store App User'): 8.05188809009119e-07,
 ('Web User', 'File & Network Sharer'): 0.0007158699064106796,
 ('Office/Productivity', 'Web User'): 3.766131469002691e-06,
 ('Office/Productivity', 'Casual Gamer'): 1.6409166939962495e-58,
 ('Office/Productivity', 'Entertainment'): 1.3102094611601196e-05,
 ('Office/Productivity', 'Communication'): 2.814925895860203e-17,
 ('Office/Productivity', 'Casual User'): 2.0317277720982354e-17,
 ('Office/Productivity', 'Gamer'): 2.0662334985985553e-49,
 ('Office/Productivity', 'Content Creator/IT'): 5.9705812657851104e-89,
 ('Office/Productivity', 'Win Store